## Considering Cost
In our first article on the [advantage of options](https://diogenesanalytics.com/blog/2024/04/05/options-advantage) we showed that the *probability of winning at least once* $P(\text{win})$ would increase (approaching *unity*) as the number of options $n$ went to infinity:

$$
\lim_{n \to \infty} P(\text{win}) = \lim_{n \to \infty} \left(1 - \prod_{i=1}^{n} P(\text{lose})_i \right) = 1
$$

But this only applies to the *probability* and does not take into consideration *the cost* of each additional option and the *payoff* upon winning. Instead if we consider the *cost* $C$ and the *payoff/utility* $U$ of winning a single option, we then arrive at an updated version of the equation introduced in the [previous article](https://diogenesanalytics.com/blog/2024/04/05/options-advantage):

$$
E(n) = (1 - p^n)U - nC
$$

What we have now is an equation that calculates the *net expected utility* (i.e. difference between the *expected utility* $(1 - p^n)U$ and the *total options cost* $nC$).

## The "Ideal" Lottery
As an example demonstration of this equation's power, let us consider an *ideal lottery* with the following criteria:
+ $U = 20,000,000$ USD is the grand prize
+ $C = 2$ USD is the cost per ticket
+ $p_w = \frac{1}{2,550,000}$ is the probability of winning $p_w$ per ticket

So then what we want to know is what value of $n$ makes this advantageous (i.e. greater than zero):

$$
E(n) = (1 - \frac{2,549,999}{2,550,000}^n) \cdot 20,000,000 - 2n > 0
$$

In [ ]:
# get enum lib
from enum import Enum

# create enum for storing lottery vars
class IdealLottery(Enum):
    U = 20 * 10 ** 6
    C = 2
    p = 1.0 - (1.0 / (2.55 * 10 ** 6))

In [ ]:
# get libs
import math
from scipy.optimize import minimize_scalar

# define the expected utility function
def expected_utility(u: float, c: float, p: float, n: int) -> float:
    """Calculates the expected utility."""
    return ((1 - p**n) * u) - (n * c)

def exp_util_derivative(u: float, c: float, p: float, n: int) -> float:
    """Calculates the derivative of the expected utility."""
    return (-u * (p**n) * math.log(p)) - c

# create custom lottery function
def calc_lottery(n: int, lotto: Enum) -> float:
    """Wrapper for lottery expected utility."""
    # get vars
    U = lotto.U.value
    C = lotto.C.value
    p = lotto.p.value

    # calculate
    return expected_utility(U, C, p, n)

def optimal_tickets() -> float:
    """Wrapper for lottery expected utility."""
    # get vars
    U = IdealLottery.U.value
    C = IdealLottery.C.value
    p = IdealLottery.p.value

    # define the negative of E(n) for minimization (since we're maximizing E(n))
    def neg_exp_util(n):
        return -expected_utility(U, C, p, n)
    
    # find the maximum point of E(n) using scipy.optimize.minimize_scalar
    result = minimize_scalar(neg_exp_util, method='bounded', bounds=(1, 10000000))

    # get optimal tickets
    return int(result.x)

In [ ]:
# get plotting libs
import matplotlib.pyplot as plt

# set the style to a dark theme
plt.style.use("dark_background")

# match website background
plt.rcParams["figure.facecolor"] = "#181818"
plt.rcParams["axes.facecolor"] = "#181818"
plt.rcParams["axes.edgecolor"] = "#181818"

# create a results dict
results = {}

# generate x/y pairs
results["expected_ideal_lotto"] = zip(*((n, calc_lottery(n, IdealLottery)) for n in range(1, 10000001, 1000)))

# Create the line plot
plt.plot(*results["expected_ideal_lotto"])

# get optimal tickets (n) / maximum E(n)
optimal_n = optimal_tickets()
maximum_E = int(expected_utility(
    IdealLottery.U.value,
    IdealLottery.C.value,
    IdealLottery.p.value,
    optimal_n
))

# plot threshold p value
plt.axvline(x=optimal_n, color=plt.cm.viridis(0.70), linestyle="--", label=f"Optimal Tickets ({optimal_n:6,d})")
plt.axhline(y=maximum_E, color=plt.cm.magma(0.90), linestyle="--", label=f"Maximum E(n) ({maximum_E:6,d})")

# add titles and labels
plt.xlabel("Number of Tickets (n)")
plt.ylabel("Expected Utility (E)")

# set title
plt.suptitle(
    "Figure 1. Optimal Number of (Ideal) Lottery Tickets", y=0.0001, fontsize=10
)

# displaying the plot
plt.legend()
plt.show()

As it turns out, this *"ideal lottery"* is quite advantageous, as you can see in *figure 1* above. There is even an optimal number of tickets $n$ where $E(n)$ (the expected utility) is maximized. This occurs at $n = 3,484,553$ and yields a maximum expected utility:

$$
\begin{align*}
    E(n) &= (1 - \frac{2,549,999}{2,550,000}^n) \cdot 20,000,000 - 2 \cdot 3,484,553 \\
    &\approx 7,930,893
\end{align*}
$$

## The REAL Lottery
But this is where the *"fantasy"* ends and the reality begins. Let us consider the same *lottery* example shown above, but this time we increase the price of each ticket $C$ to $100$ USD:
+ $U = 20,000,000$ USD is the grand prize
+ $C = 100$ USD is the cost per ticket
+ $p_w = \frac{1}{2,550,000}$ is the probability of winning $p_w$ per ticket

So what would we expect to happen now?

In [ ]:
# define enum for real lottery
class RealLottery(Enum):
    U = 20 * 10 ** 6
    C = 100
    p = 1.0 - (1.0 / (2.55 * 10 ** 6))

In [ ]:
# generate x/y pairs
results["expected_real_lotto"] = zip(*((n, calc_lottery(n, RealLottery)) for n in range(1, 10)))

# Create the line plot
plt.plot(*results["expected_real_lotto"])

# add titles and labels
plt.xlabel("Number of Tickets (n)")
plt.ylabel("Expected Utility (E)")

# set title
plt.suptitle(
    "Figure 2. Real Lottery Expected Utility", y=0.0001, fontsize=10
)

# displaying the plot
plt.show()

What we see is that basically the *expected utility* is never going to be positive (i.e. you are *losing* instead of *winning*). Every ticket you buy costs you more than what you can *"expect"* to win. This is the **power** of the cost $C$ of each additional option. In the previous *"ideal"* lottery, it was only the lower cost of $2$ USD that made it *advantageous* to buy ANY number of tickets. Now with the price set to $100$ USD, there is no real *winning* ... only losing.

## Moral
While it is certainly true that buying more tickets for the *lottery* will increase your probability of winning, but it will **NOT** increase your *expected utility* and hence your profit. In fact you will be in a *worse* position after buying $n >= 1$ tickets, because you will now be $100 \cdot n$ less wealthy. This example perfectly illustrates why only considering the *probability* of winning, and not factoring in the *utility* is an ultimately incomplete calculation. *Utility* must be considered for any situation where *advantage* is the desired calculation.